In [ ]:
library(tidyverse)
library(nycflights13)

# Lecture 9: Functions (Lecture 14)

In this lecture we will learn about functions. We already have plenty of experience calling functions in order to mutate, plot, and explore data. Now we will learn how and when to write our own functions.

## Functions

Recall that we can write functions when we find ourselves repeating the same operations multiple times.

In [ ]:
df = tibble::tibble(
  a = rnorm(10),
  b = rnorm(10),
  c = rnorm(10),
  d = rnorm(10)
)

print(df)

In [ ]:
df$a = (df$a - min(df$a, na.rm = TRUE)) / 
    (max(df$a, na.rm = TRUE) - min(df$a, na.rm = TRUE))
df$b = (df$b - min(df$b, na.rm = TRUE)) / 
    (max(df$b, na.rm = TRUE) - min(df$b, na.rm = TRUE))
df$c = (df$c - min(df$c, na.rm = TRUE)) / 
    (max(df$c, na.rm = TRUE) - min(df$c, na.rm = TRUE))
df$d = (df$d - min(df$d, na.rm = TRUE)) / 
    (max(df$d, na.rm = TRUE) - min(df$d, na.rm = TRUE))
 
print(df)

In [ ]:
rescale01 = function(x) {
    (x - min(x, na.rm = TRUE)) / (max(x, na.rm = TRUE) - min(x, na.rm = TRUE))
}

df$a = rescale01(df$a)
df$b = rescale01(df$b)
df$c = rescale01(df$c)
df$d = rescale01(df$d)

print(df)

## Conditional execution
Often when writing functions we need to do different things depending on what data is passed in. This is known as *conditional execution*, and is accomplished using the `if/else` construct:
```{r}
if (condition) {
  # code executed when condition is TRUE
} else {
  # code executed when condition is FALSE
}
```

### Example
The *Heaviside step function* is defined as
$$H(x)=\begin{cases}0,&x\le 0\\
1,&\text{otherwise}
\end{cases}.$$
How can we code this as an R function?

In [ ]:
H = function(x) {
    if (x <= 0) {
        return(0)
    } else {
        return(1)
    }
}
H(-1)
H(0)
H(1)

In [ ]:
H2 = function(x) {
    ifelse(x <= 0, 0, 1)
}

H2(c(-1, 0, 1))

### Multiple conditions
Sometimes you will want to check multiple conditions using an `if` statement. For example, let's define the function $$\operatorname{sgn}(x) = \begin{cases}-1,&x<0\\0,&x=0\\1,&x>0.\end{cases}$$

In [ ]:
sgn = function(x) {
    # code
}

The general form is
```{r}
if (this) {
  # do that
} else if (that) {
  # do something else
} else {
  # 
}
```

If you find yourself chaining together a long string of `if/else if/else if/else if/.../else` statements, chances are there is an easier way. For example, say we have a continuous variable `temp` that we want to convert to a factor. One way is using lots of `if/else`:
```{r}
if (temp <= 0) {
  "freezing"
} else if (temp <= 10) {
  "cold"
} else if (temp <= 20) {
  "cool"
} else if (temp <= 30) {
  "warm"
} else {
  "hot"
}
```

Alternatively, we could use the `cut` function:

In [ ]:
cut(10, c(-Inf, 0, 10, 20, 30, Inf), c('freezing', 'cold', 'cool', 'warm', 'hot'))

### Brackets
Both `function` and `if` are usually called using the curly bracket delimiters `{` and `}`. For one-line statements, the brackets are optional:

In [ ]:
if (TRUE) { 
    print("A1") 
} else { 
    print("B") 
}
if (TRUE) print("A2") else print("B")

You should almost always use the curly braces. One exception is for very brief, unnamed functions. We'll see some examples of this next week when we study map/reduce computations.

## Function arguments
Functions can take multiple arguments. Generally they fall into one of two categories:
* *Data* to be processed by the function, and
* *Options*, which affect how the data gets processed.

```{r}
mean(x, na.rm=TRUE)
log(x, base=y)
str_c(..., sep=" ")
```
What is/are the data? What are the options?

### Rules for function arguments
Generally:
1. The *data* parameters should come first; and
2. The *options* should come second, and have sensible defaults.

Default parameter values are specified by the `option=default` notation:

In [ ]:
mean_ci = function(x, conf=0.95) {
  se = sd(x) / sqrt(length(x))
  alpha = 1 - conf
  mean(x) + se * qnorm(c(alpha / 2, 1 - alpha / 2))
}

When you call a function, you can omit the values of the default arguments. If overriding the default, you should specify the parameter you are overriding and then input the overridden value with an ` = ` in between:
```{r}
mean_ci(c(1, 2, 3, 4))              # uses default value
mean_ci(c(1, 2, 3, 4), conf = .99)  # ok
mean_ci(c(1, 2, 3, 4), conf=.99)    # ok
mean_ci(c(1, 2, 3, 4), .99)         # dangerous
```

### Validation
When writing functions it's a good idea to *validate* the input -- that is, make sure it matches your assumptions about what is being passed to the function. Consider the following function which returns the weighted average of a vector:

In [ ]:
w_mean = function(x, w) {
    (x * w) / sum(w)
}

This function relies implicitly on the fact that the weight vector `w` is the same length as the input vector `x`. If it's not, you'll get a warning and unexpected behavior.

In [ ]:
w_mean(c(1,2,3), w=c(1, 2))

It's best to make the assumption of equal length explicit by checking it:

In [ ]:
w_mean = function(x, w) {
    stopifnot(length(w) == length(x))
    (x * w) / sum(w)
}

Now:

In [ ]:
w_mean(c(1,2,3), w=c(1, 2))

Adding comments is another good way to make sure that you don't encounter unexpected situations in your functions:

In [ ]:
w_mean = function(x, w) {
    # Return the average of `x` weighted by weight vector `w`
    stopifnot(length(w) == length(x))
    (x * w) / sum(w)
}

###  Dot-dot-dot (`…`)
Some functions are designed to take a variable number of inputs. We saw this for example with the `str_c` function:

In [ ]:
stringr::str_c("a", "b")
stringr::str_c("a", "b", "c", "d")

To construct a function that takes a variable number of arguments we use the `...` notation:
```{r}
f = function(...) {
    <do something with variable arguments>
}
```

One thing you can do with the `...` is pass it to another function:

In [ ]:
commas <- function(...) stringr::str_c(..., collapse = ", ")
commas(letters[1:10])

You can also access individual arguments in `...` using the `list(...)` notation. We'll learn more about lists in the next lecture.

## Return values
Thus far we have relied on the default behavior of R, which is to return the last value in the function:

In [ ]:
f = function() {
    1
    2
    3  # this will be returned
}
f()

In more complicated functions you'll need to manually return values using the `return()` function:
```{r}
complicated_function <- function(x, y, z) {
  if (length(x) == 0 || length(y) == 0) {
      return(0)  # this immediately returns and halts the function.
  }
  # Complicated code here
}
```

### Pipeable functions
We've seen a lot of uses of the pipe operator `%>%`. As you become more advanced, you may find it useful to create your own functions which can be used in data pipelines. 

#### Transformations
For pipeable functions that transform a data frame, simply return the altered version of the data frame. For example:

In [ ]:
first_row <- function(df) {
    df %>% slice(1)
}
tibble(x=c(1,2,3), y=c("a","b","c")) %>% first_row

### Challenge problem
Define a function `drop_even()` which drops all the even-numbered rows from a data frame:
```{r}
> tibble(x=c(1,2,3), y=c("a","b","c")) %>% drop_even
# A tibble: 2 x 2
      x y    
  <int> <chr>
1     1 a    
2     3 c    
```

In [ ]:
drop_even = function(df) {
    # drop all the even-numbered observations
}

#### Side effects
Some functions have *side effects* but don't modify the original data frame. For example, consider the following function which counts how many missing values are present in a data frame:

In [ ]:
show_missings = function(df) {
  n <- sum(is.na(df))
  cat("Missing values: ", n, "\n", sep = "")
  df  # note return value
}

This function works but has the undesirable effect of printing the whole data frame when it returns:

In [ ]:
show_missings(mpg)

To correct this tidyverse has the `invisible()` function:

In [ ]:
show_missings = function(df) {
  n <- sum(is.na(df))
  cat("Missing values: ", n, "\n", sep = "")
  invisible(df)  # return will not print out
}

Now we can run the command interactively, and also use it in pipelines:

In [ ]:
show_missings(flights)

In [ ]:
flights %>% filter(month < 5) %>% show_missings

## Environments
The environment is, roughly, the set of variables and data defined in your R session. The default environment is called the "global environment":

In [ ]:
environment()

A function depends on the environment in which it was defined. In particular, if you reference a variable inside of a function, which is not *defined* in that function, R will look for it in the enclosing environment.

In [ ]:
f = function(x) {
    x + y
}
x = 1:3
y = 3
f(x)

In [ ]:
y = 4
f(x)

In [ ]:
f = function(x) {
    y = x
    print(y)
}

y = 10
f(5)
y

In [ ]:
f = function(x) {
    y <<- x
    print(y)
}

y = 10
f(5)
y

### Challenge problem
Write a function `howmany()` which prints the number of times that it has been called:
```
> howmany()
[1] 1
> howmany()
[1] 2
> howmany()
[1] 3
```



Harder: write a function `same(x)` which prints "yes!" if `x` is the same value that was passed into `same()` on the previous call, and "no!" otherwise. (`same()` always prints "no!" to start):
```{r}
> same(1)
[1] "no!"
> same(1)
[1] "yes!"
> same(1)
[1] "yes!"
> same("hello")
[1] "no!"
```